## Deployer

In [2]:
%pip install azureml-sdk

  Using cached azureml_sdk-1.60.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached azureml_dataset_runtime-1.60.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached azureml_train_core-1.60.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached azureml_train_automl_client-1.60.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached azureml_pipeline-1.60.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached azureml_dataprep-5.1.6-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyarrow-19.0.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached fusepy-3.0.1.tar.gz (11 kB)
  Preparing metadata (setup.py) ... done
  Using cached azureml_pipeline_core-1.60.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached azureml_pipeline_steps-1.60.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached azureml_automl_core-1.60.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached azureml_telemetry-1.60.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached azureml_train_restclients_hyperdrive-1.60.0-py3-no

In [3]:
%pip install azureml
%pip install azureml-core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Creando el workspace:

In [32]:
import json
from azureml.core.model import Model
from azureml.core import Workspace

# Cargando el archivo con el ID
id = open('id_conf_c4.json', 'r')
mi = json.load(id)
my_id = mi["id"]

# Creando el workspace llamando el ID
ws = Workspace.create(name="worksp_cl_4",
                      subscription_id = my_id,
                      resource_group = "class4",
                      location = "centralindia")

# Registrando el modelo
model_name = "model_regression"
registered_model = Model.register(model_path="model_regression.pkl",
                                  model_name=model_name,
                                  workspace=ws)

Deploying AppInsights with name workspclinsights0c42dac4.
Deployed AppInsights with name workspclinsights0c42dac4. Took 12.03 seconds.
Deploying StorageAccount with name workspclstorage4ad2d7209.
Deploying KeyVault with name workspclkeyvault593444ee.
Deployed KeyVault with name workspclkeyvault593444ee. Took 37.69 seconds.
Deployed StorageAccount with name workspclstorage4ad2d7209. Took 31.7 seconds.
Deploying Workspace with name worksp_cl_4.
Deployed Workspace with name worksp_cl_4. Took 44.0 seconds.
Registering model model_regression


In [33]:
################ score.py ###################

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model
import re

def init():
  global model
  model_path = Model.get_model_path('{model_name}')
  model = joblib.load(model_path)

def run(raw_data):
  try: 
    if isinstance(raw_data, str):
      data_dict = json.loads(raw_data)
    else:
      data_dict = raw_data
    
    # Convertir directamente a DataFrame
    data = pd.DataFrame(data_dict["data"])
    
    # Incluyendo el preprocesamiento de datos
    
    cols_cat = [
      "NameStyle",
      "Title",
      "FirstName",
      "MiddleName",
      "LastName",
      "Suffix",
      "CompanyName",
      "SalesPerson",
      "EmailAddress",
      "PasswordHash",
      "rowguid",
      "PasswordSalt"]
    
    data = data.drop(columns=[col for col in cols_cat if col in data.columns], errors='ignore')
    
    data['Phone'] = data['Phone'].astype(str).str.replace(r'\-', '', regex=True)
    data['Phone'] = data['Phone'].str.replace(r'\s+', '', regex=True)
    data['Phone'] = data['Phone'].str.replace(r'\(', '', regex=True)
    data['Phone'] = data['Phone'].str.replace(r'\)', '', regex=True)
    data['Phone'] = data['Phone'].str.replace(r'\+', '', regex=True)
    data['Phone'] = data['Phone'].astype(np.float64)
    
    data = data.fillna(0) # Rellenando los valores nulos con 0
     
    # Realizando la predicción
    no_format_result = model.predict(data).tolist() # Valores predichos sin formato
    
    result = pd.to_datetime(no_format_result, unit = 's') # Valores predichos con formato
    final_result = result.strftime('%Y-%m-%d %H:%M:%S').tolist()

    return json.dumps(final_result) 
    
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [34]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configurando el entorno de virtual
v_env = Environment("regression_env")
v_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'numpy'])

# Configurando el entorno de inferencia
inference_config = InferenceConfig(
                                environment=v_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) 

# Construyendo el servicio
service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

/var/folders/by/zjps0pld1k3grf79nwzwkw440000gn/T/ipykernel_82200/2259524308.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [35]:
# Esperando a que el servicio se despliegue
service.wait_for_deployment()

# Guardando la URI del servicio
scoring_uri = service.scoring_uri

# Guardando la URI en un archivo JSON
scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
